# **Compliance GPT with LangChain**

## **Setup**

In [1]:
from dotenv import load_dotenv
load_dotenv()

True

## **Config**

In [2]:
from utils.config import get_config
from utils.models import ModelName, get_model

config = get_config()

In [7]:
USER_ID = 'xmriz'
CONVERSATION_ID = 'xmriz-2021-07-01-01'

## **Define Model**

In [3]:
model_name = ModelName.AZURE_OPENAI
llm_model, embed_model = get_model(model_name=model_name, config=config)

## **Load Vector Store Index**

In [4]:
from databases.vector_store import RedisIndexManager

redis = RedisIndexManager(index_name='ojk', embed_model=embed_model, config=config)
vector_store = redis.load_vector_index()

c:\Users\acer\miniconda3\envs\chatbot-all\lib\site-packages\pinecone\data\index.py:1: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


## **Get Retriever**

In [5]:
from retriever.retriever_ojk import get_retriever_ojk

retriever = get_retriever_ojk(vector_store=vector_store, top_n=7, top_k=20, llm_model=llm_model, embed_model=embed_model, config=config)

## **Create Chain**

In [6]:
from constant.ojk.prompt import CONTEXTUALIZE_Q_PROMPT_STR, QA_SYSTEM_PROMPT_STR
from databases.chat_store import MongoDBChatStore, RedisChatStore
from rag_chain import create_chain_with_chat_history

chat_store = RedisChatStore(k=5, config=config)

chain = create_chain_with_chat_history(
    contextualize_q_prompt_str=CONTEXTUALIZE_Q_PROMPT_STR,
    qa_system_prompt_str=QA_SYSTEM_PROMPT_STR,
    retriever=retriever,
    llm_model=llm_model,
    chat_store=chat_store
)

## **Invoke**

#### **With Question, Answer, and Context**

In [19]:
from rag_chain import get_response

response = get_response(
    chain=chain,
    question="Apa peraturan tentang pasar modal?",
    user_id=USER_ID,
    conversation_id=CONVERSATION_ID
)

response

{'rewrited question': 'Apa yang diatur oleh peraturan pasar modal?',
 'answer': 'Peraturan pasar modal mengatur tentang ketentuan umum, dasar penilaian investasi dana pensiun, dan ketentuan',
 'context': '[\n  {\n    "metadata": {\n      "id": "doc:ojk:267dadf3bfc44d2bb31fb2075c7d6c9c",\n      "title": "Dasar Penilaian Investasi Dana Pensiun",\n      "sector": "IKNB",\n      "subsector": "Dana Pensiun",\n      "regulation_type": "Surat Edaran OJK",\n      "regulation_number": "4/SEOJK.05/2024",\n      "effective_date": "1 Juli 2024",\n      "file_url": "https://www.ojk.go.id/id/regulasi/Documents/Pages/Dasar-Penilaian-Investasi-Dana-Pensiun/FAQ%20SEOJK%204-SEOJK.05-2024%20Dasar%20Penilaian%20Investasi%20Dana%20Pensiun.pdf",\n      "doc_id": "3",\n      "page_number": "1",\n      "relevance_score": 0.2791402\n    },\n    "page_content": "2. \\nApa saja yang diatur dalam SEOJK ini? \\nPokok-pokok perubahan pengaturan dalam SEOJK ini meliputi: \\n1. \\nKetentuan Umum \\n2. \\nDasar Penila

#### **Streaming**

In [20]:
from rag_chain import print_answer_stream

print_answer_stream(chain=chain, question="Siapa itu lionel messi?", user_id=USER_ID, conversation_id=CONVERSATION_ID)

Maaf, pertanyaan yang Anda ajukan tidak terkait dengan konteks dan informasi metadata yang diberikan. Silakan ajukan pertanyaan lain yang terkait dengan kepatuhan perbankan di Indonesia.

#### **Chat Store Monitor**

In [21]:
# to print chat history
print(chat_store.get_session_history(user_id=USER_ID, conversation_id=CONVERSATION_ID))

Human: Apa peraturan tentang pasar modal?
AI: Peraturan pasar modal mengatur tentang ketentuan umum, dasar penilaian investasi dana pensiun, dan ketentuan
Human: Siapa itu lionel messi?
AI: Maaf, pertanyaan yang Anda ajukan tidak terkait dengan konteks dan informasi metadata yang diberikan. Silakan ajukan pertanyaan lain yang terkait dengan kepatuhan perbankan di Indonesia.


In [17]:
# to make the chat history empty
chat_store.clear_all()